# First pass filter

This script 
1.Selects the wanted dimensions
2.Filters 
    a.MCC/MNC = 234/20 (Three UK)
    b.GPS accuracy ≤ 10m
    c.Valid signal strength (RSRP < 0)   
3.Adds Cartesian coordinates
4.Aggregates the data into calendar monthly files for subsequent processing.


Originally developed using the PyCharm IDE but converted to Jupyter Notebook
Version 0.1 (16/1/21) First version. Writes monthly summaries of the wanted values
Version 0.2 (14/2/21) Removed some columns and also rows north of 600km (still captures 93% of the population)

In [1]:
import pandas as pd
import glob
from pyproj import Transformer

import xlrd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.formula.api as smf

In [2]:
# define a directory to write any outputs
workdir = 'C:/1drive/OneDrive - Three/_avado_Masters/_PROJECT/plots/'
# define location of the raw data
ookladir = 'P:/ookla/'

In [3]:
# After checking the metadata descriptions and looking at the data, these are the columns which appeared to be most useful
cols = [3, 19, 20, 25, 26, 27, 30, 31, 33, 34, 37, 59, 64, 95, 96, 101, 102, 103, 104, 112, 113, 115, 116, 120,
         121, 122, 126, 127, 128, 133, 134, 136, 141, 142]

In [4]:
# The data is provided in daily files. After filtering for just Three subscribers concatenating these by calendar 
# month results in aggregated files containing about 500k records per month
# Create an array of months to iterate over
mths = ['2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02']

In [5]:
# This define the projections and transformation method for converting between polar ('lat/long') coordinates and
# Cartesian (Eastings/Northings)
bng = 'epsg:27700'
wgs84 = 'epsg:4326'
transformer = Transformer.from_crs(wgs84, bng)

In [6]:
"""
Iterate over each of the months to load the daily files (required dimensions only), filter as per the 'requirements'
stated above, add the location in Cartesian form, aggregate by calendar month and rewrite as a monthly CSV
This section does all the heavy lifting and takes about 10 minutes per month to run
"""
for m in mths:
    android_zips = glob.glob(ookladir + 'android_' + m + '*.zip')
    # print(android_zips)
    # create an empty dataframe
    android_df = pd.DataFrame
    for zipf in android_zips:
        print('checking', zipf)
        ook = pd.read_csv(zipf, compression='zip', header=0, usecols=cols, low_memory=False)
        print(ook.__len__(), 'records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres')
        ook = ook[ook['mnc'] == 20]
        ook = ook[ook['mcc'] == 234]
        # coerce the location accuracy and rsrp columns to integers
        ook.location_accuracy_a = pd.to_numeric(ook.location_accuracy_a, errors='coerce').fillna(0).astype(np.int64)
        ook.rsrp_a = pd.to_numeric(ook.rsrp_a, errors='coerce').fillna(0).astype(np.int64)
        ook = ook[ook['location_accuracy_a'] <= 10]
        # rsrp should always be negative so selecting where this is less than zero will remove missing values
        ook = ook[ook['rsrp_a'] < 0]
        print(ook.__len__(), 'records remain')
        # appending to an empty dataframe doesn't work, get around this with try/except
        # first time try will fail and the except initialises the dataframe with contents of the first zip file
        # subsequent attempts the try will append the contents of the other zip file
        try:
            android_df = android_df.append(ook)
        except:
            android_df = ook

        print('cumulative record count', android_df.__len__(), 'records')

    print('Adding Eastings and Northings ...')
    android_df.reset_index(inplace=True)
    for i, row in android_df.iterrows():
        # print(i, '>>', row)
        E, N = transformer.transform(row['client_latitude'], row['client_longitude'])
        # print(i, '>> ', E, ':  ', N)
        android_df.loc[i, 'E'] = E
        android_df.loc[i, 'N'] = N
        if i % 100 == 0:
            print(i, 'of', android_df.__len__(), 'rows processed')

    # remove rows further north than (roughly) Newcastle (600km)
    android_df = android_df[android_df['N'] < 600000]
    # write this month's transformed file to csv.
    android_df.to_csv(ookladir + m + '.csv')


    
# (The logging is quite verbose but confirms it's running ok!)

checking P:/ookla\android_2020-05-23.zip
81839 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5432 records remain
cumulative record count 5432 records
checking P:/ookla\android_2020-05-24.zip
71264 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5181 records remain
cumulative record count 10613 records
checking P:/ookla\android_2020-05-25.zip
66557 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
4966 records remain
cumulative record count 15579 records
checking P:/ookla\android_2020-05-26.zip
76125 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5195 records remain
cumulative record count 20774 records
checking P:/ookla\android_2020-05-27.zip
76076 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5338 records remain
cumulative record count 26112 records
checking P:/ookla\android_2020-05-28.zip
74616 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5208 records remain
cumulative record count 3

22900 of 46810 rows processed
23000 of 46810 rows processed
23100 of 46810 rows processed
23200 of 46810 rows processed
23300 of 46810 rows processed
23400 of 46810 rows processed
23500 of 46810 rows processed
23600 of 46810 rows processed
23700 of 46810 rows processed
23800 of 46810 rows processed
23900 of 46810 rows processed
24000 of 46810 rows processed
24100 of 46810 rows processed
24200 of 46810 rows processed
24300 of 46810 rows processed
24400 of 46810 rows processed
24500 of 46810 rows processed
24600 of 46810 rows processed
24700 of 46810 rows processed
24800 of 46810 rows processed
24900 of 46810 rows processed
25000 of 46810 rows processed
25100 of 46810 rows processed
25200 of 46810 rows processed
25300 of 46810 rows processed
25400 of 46810 rows processed
25500 of 46810 rows processed
25600 of 46810 rows processed
25700 of 46810 rows processed
25800 of 46810 rows processed
25900 of 46810 rows processed
26000 of 46810 rows processed
26100 of 46810 rows processed
26200 of 4

71061 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5001 records remain
cumulative record count 37650 records
checking P:/ookla\android_2020-06-08.zip
73647 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5139 records remain
cumulative record count 42789 records
checking P:/ookla\android_2020-06-09.zip
78593 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5447 records remain
cumulative record count 48236 records
checking P:/ookla\android_2020-06-10.zip
84560 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5602 records remain
cumulative record count 53838 records
checking P:/ookla\android_2020-06-11.zip
94993 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
6331 records remain
cumulative record count 60169 records
checking P:/ookla\android_2020-06-12.zip
86175 records read ... selecting mnc/mcc 234/20 accuracy <= 10 metres
5540 records remain
cumulative record count 65709 records
checking P:/ookla\android_2

13700 of 160137 rows processed
13800 of 160137 rows processed
13900 of 160137 rows processed
14000 of 160137 rows processed
14100 of 160137 rows processed
14200 of 160137 rows processed
14300 of 160137 rows processed
14400 of 160137 rows processed
14500 of 160137 rows processed
14600 of 160137 rows processed
14700 of 160137 rows processed
14800 of 160137 rows processed
14900 of 160137 rows processed
15000 of 160137 rows processed
15100 of 160137 rows processed
15200 of 160137 rows processed
15300 of 160137 rows processed
15400 of 160137 rows processed
15500 of 160137 rows processed
15600 of 160137 rows processed
15700 of 160137 rows processed
15800 of 160137 rows processed
15900 of 160137 rows processed
16000 of 160137 rows processed
16100 of 160137 rows processed
16200 of 160137 rows processed
16300 of 160137 rows processed
16400 of 160137 rows processed
16500 of 160137 rows processed
16600 of 160137 rows processed
16700 of 160137 rows processed
16800 of 160137 rows processed
16900 of

40200 of 160137 rows processed
40300 of 160137 rows processed
40400 of 160137 rows processed
40500 of 160137 rows processed
40600 of 160137 rows processed
40700 of 160137 rows processed
40800 of 160137 rows processed
40900 of 160137 rows processed
41000 of 160137 rows processed
41100 of 160137 rows processed
41200 of 160137 rows processed
41300 of 160137 rows processed
41400 of 160137 rows processed
41500 of 160137 rows processed
41600 of 160137 rows processed
41700 of 160137 rows processed
41800 of 160137 rows processed
41900 of 160137 rows processed
42000 of 160137 rows processed
42100 of 160137 rows processed
42200 of 160137 rows processed
42300 of 160137 rows processed
42400 of 160137 rows processed
42500 of 160137 rows processed
42600 of 160137 rows processed
42700 of 160137 rows processed
42800 of 160137 rows processed
42900 of 160137 rows processed
43000 of 160137 rows processed
43100 of 160137 rows processed
43200 of 160137 rows processed
43300 of 160137 rows processed
43400 of

...

4000 of 124057 rows processed
4100 of 124057 rows processed
4200 of 124057 rows processed
4300 of 124057 rows processed
4400 of 124057 rows processed
4500 of 124057 rows processed
4600 of 124057 rows processed
4700 of 124057 rows processed
4800 of 124057 rows processed
4900 of 124057 rows processed
5000 of 124057 rows processed
5100 of 124057 rows processed
5200 of 124057 rows processed
5300 of 124057 rows processed
5400 of 124057 rows processed
5500 of 124057 rows processed
5600 of 124057 rows processed
5700 of 124057 rows processed
5800 of 124057 rows processed
5900 of 124057 rows processed
6000 of 124057 rows processed
6100 of 124057 rows processed
6200 of 124057 rows processed
6300 of 124057 rows processed
6400 of 124057 rows processed
6500 of 124057 rows processed
6600 of 124057 rows processed
6700 of 124057 rows processed
6800 of 124057 rows processed
6900 of 124057 rows processed
7000 of 124057 rows processed
7100 of 124057 rows processed
7200 of 124057 rows processed
7300 of 12

30800 of 124057 rows processed
30900 of 124057 rows processed
31000 of 124057 rows processed
31100 of 124057 rows processed
31200 of 124057 rows processed
31300 of 124057 rows processed
31400 of 124057 rows processed
31500 of 124057 rows processed
31600 of 124057 rows processed
31700 of 124057 rows processed
31800 of 124057 rows processed
31900 of 124057 rows processed
32000 of 124057 rows processed
32100 of 124057 rows processed
32200 of 124057 rows processed
32300 of 124057 rows processed
32400 of 124057 rows processed
32500 of 124057 rows processed
32600 of 124057 rows processed
32700 of 124057 rows processed
32800 of 124057 rows processed
32900 of 124057 rows processed
33000 of 124057 rows processed
33100 of 124057 rows processed
33200 of 124057 rows processed
33300 of 124057 rows processed
33400 of 124057 rows processed
33500 of 124057 rows processed
33600 of 124057 rows processed
33700 of 124057 rows processed
33800 of 124057 rows processed
33900 of 124057 rows processed
34000 of

57400 of 124057 rows processed
57500 of 124057 rows processed
57600 of 124057 rows processed
57700 of 124057 rows processed
57800 of 124057 rows processed
57900 of 124057 rows processed
58000 of 124057 rows processed
58100 of 124057 rows processed
58200 of 124057 rows processed
58300 of 124057 rows processed
58400 of 124057 rows processed
58500 of 124057 rows processed
58600 of 124057 rows processed
58700 of 124057 rows processed
58800 of 124057 rows processed
58900 of 124057 rows processed
59000 of 124057 rows processed
59100 of 124057 rows processed
59200 of 124057 rows processed
59300 of 124057 rows processed
59400 of 124057 rows processed
59500 of 124057 rows processed
59600 of 124057 rows processed
59700 of 124057 rows processed
59800 of 124057 rows processed
59900 of 124057 rows processed
60000 of 124057 rows processed
60100 of 124057 rows processed
60200 of 124057 rows processed
60300 of 124057 rows processed
60400 of 124057 rows processed
60500 of 124057 rows processed
60600 of

84200 of 124057 rows processed
84300 of 124057 rows processed
84400 of 124057 rows processed
84500 of 124057 rows processed
84600 of 124057 rows processed
84700 of 124057 rows processed
84800 of 124057 rows processed
84900 of 124057 rows processed
85000 of 124057 rows processed
85100 of 124057 rows processed
85200 of 124057 rows processed
85300 of 124057 rows processed
85400 of 124057 rows processed
85500 of 124057 rows processed
85600 of 124057 rows processed
85700 of 124057 rows processed
85800 of 124057 rows processed
85900 of 124057 rows processed
86000 of 124057 rows processed
86100 of 124057 rows processed
86200 of 124057 rows processed
86300 of 124057 rows processed
86400 of 124057 rows processed
86500 of 124057 rows processed
86600 of 124057 rows processed
86700 of 124057 rows processed
86800 of 124057 rows processed
86900 of 124057 rows processed
87000 of 124057 rows processed
87100 of 124057 rows processed
87200 of 124057 rows processed
87300 of 124057 rows processed
87400 of

110400 of 124057 rows processed
110500 of 124057 rows processed
110600 of 124057 rows processed
110700 of 124057 rows processed
110800 of 124057 rows processed
110900 of 124057 rows processed
111000 of 124057 rows processed
111100 of 124057 rows processed
111200 of 124057 rows processed
111300 of 124057 rows processed
111400 of 124057 rows processed
111500 of 124057 rows processed
111600 of 124057 rows processed
111700 of 124057 rows processed
111800 of 124057 rows processed
111900 of 124057 rows processed
112000 of 124057 rows processed
112100 of 124057 rows processed
112200 of 124057 rows processed
112300 of 124057 rows processed
112400 of 124057 rows processed
112500 of 124057 rows processed
112600 of 124057 rows processed
112700 of 124057 rows processed
112800 of 124057 rows processed
112900 of 124057 rows processed
113000 of 124057 rows processed
113100 of 124057 rows processed
113200 of 124057 rows processed
113300 of 124057 rows processed
113400 of 124057 rows processed
113500 o